<a href="https://colab.research.google.com/github/Decoding-Data-Science/nov25/blob/main/crewai_multiagent_level_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gitex workshop: Create Multi Agents to Research and Write an Article

In this workshop, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

https://huggingface.co/spaces/decodingdatascience/fromprompttopost

In [8]:
#install packages
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [9]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [10]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.



In [11]:
#setting the model
import os

# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-nano-2025-04-14'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [12]:
# Retrieve API key from Colab secrets and set it in environment variables
from dotenv import load_dotenv
load_dotenv()  # <-- THIS loads the .env file automatically

api_key=os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key

In [13]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

In [14]:
# Set up API keys - enter from https://serper.dev/billing
os.environ["SERPER_API_KEY"] = "218873db9b73d345d3083e2f1f29cb74b14d4ddb" # serper.dev API key

# Instantiate tools
# docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

In [16]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."    ,
    tools=[search_tool, web_rag_tool],
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [17]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic in 1500 words: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [18]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [19]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [20]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.1500 words",
    agent=writer,
)

### Task: Edit

In [21]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [22]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    planning=True,
    verbose=False
)


**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

## Running the Crew
## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [23]:
topic = "GPT 5.2"
result = crew.kickoff(inputs={"topic": topic})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on GPT 5.2.                            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.1. Start by using the 'Search the internet with Serper'   │
│  tool to gather the latest articles and news about GPT 5.2. Use search queries like 'GPT 5.2 trends', 'GPT 5.2  │
│  updates', 'key players in GPT 5.2 development', and 'news on GPT 5.2'.                                         │
│                                                                                                                 │
│  2. Review the gathered articles to extract significant trends, emerging key players in the market, and         │
│  noteworthy news insights. Look for repetitive themes or new technologies that could impact GPT 5.2.            │
│                                                                                                                 │
│  3. Create a summary document detailing these findings, formatted as a bullet list to highlight each trend,     │
│  key player, and noteworthy news to enhance clarity.                                                            │
│                                                                                                                 │
│  4. Organize this document into sections that categorize trends, players, and news, ensuring each point is      │
│  backed by data from the sources collected.                                                                     │
│                                                                                                                 │
│  5. Conclude with a brief reflection on how these trends could influence content creation and user engagement   │
│  with GPT 5.2.                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Action: Search the internet with Serper                                                                        │
│  Search Query: GPT 5.2 trends                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on GPT 5.2.                                      │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│  1. Begin by reviewing the content outline developed in Task 1, focusing on defining the target audience,       │
│  their interests, and pain points relevant to GPT 5.2.                                                          │
│                                                                                                                 │
│  2. Write a compelling introduction that captures the reader's attention and introduces the relevance of GPT    │
│  5.2 in today's climate. Mention key trends identified to set the context.                                      │
│                                                                                                                 │
│  3. Structure the blog post into sections based on the content outline, using engaging subtitles for each       │
│  section. Each section should contain 2-3 well-developed paragraphs that elaborate on key points identified in  │
│  the outline while integrating SEO keywords naturally.                                                          │
│                                                                                                                 │
│  4. Conclude the blog post with a strong call to action that encourages reader interaction, whether it's to     │
│  share their thoughts on GPT 5.2 or explore further content.                                                    │
│                                                                                                                 │
│  5. Ensure that the blog post flows well with seamless transitions between sections. Keep sentences concise     │
│  and avoid jargon unless explained for audience understanding.                                                  │
│                                                                                                                 │
│  6. Use markdown formatting for headers, bullet points, and links to enhance readability and engage readers     │
│  visually.                                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Unlocking the Future: A Deep Dive into GPT 5.2 and Its Impact on AI Innovation                               │
│                                                                                                                 │
│  ## Introduction: Navigating the New Dawn of Artificial Intelligence                                            │
│                                                                                                                 │
│  In recent years, artificial intelligence has transitioned from a niche technological pursuit to a central      │
│  pillar of modern innovation. The rapid evolution of language models, particularly the GPT series developed by  │
│  OpenAI, exemplifies this exponential growth. As the AI community anticipates the latest iteration—GPT          │
│  5.2—discussions around its potential capabilities, implications, and applications are intensifying. This       │
│  model promises to push the boundaries of what AI can achieve, integrating cutting-edge advancements and        │
│  addressing previous limitations.                                                                               │
│                                                                                                                 │
│  Current trends point toward ever-increasing adoption of AI across industries—from healthcare and finance to    │
│  entertainment and education. As businesses and developers explore the capabilities of GPT 5.2, understanding   │
│  its features, improvements, and potential challenges becomes crucial. Whether you're a tech enthusiast, a      │
│  business leader, or a researcher, staying informed about GPT 5.2 is vital to harnessing its full potential.    │
│  This piece aims to unpack the significance of GPT 5.2, exploring why it matters now more than ever, and what   │
│  future innovations it may unlock in the realm of AI.                                                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## The Evolution of GPT Models: From GPT-3 to GPT 5.2                                                          │
│                                                                                                                 │
│  The journey from GPT-3 to GPT 5.2 underscores a trajectory of continuous improvement driven by advances in     │
│  neural network architecture, training data scale, and computational power. GPT-3, launched in 2020, set a new  │
│  standard with 175 billion parameters, demonstrating remarkable language understanding and generation           │
│  abilities. Its release opened floodgates for AI integration into chatbots, content creation, and automation    │
│  tools, but also exposed limitations such as contextual misunderstandings and biases.                           │
│                                                                                                                 │
│  By the time GPT 4 was launched, expectations had skyrocketed. The model showcased strides in natural language  │
│  processing (NLP), with enhanced contextual awareness a

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.1. Begin by    │
│  reading through the entire blog post to understand the overall narrative and ensure it aligns with the         │
│  brand's voice and tone.                                                                                        │
│                                                                                                                 │
│  2. Check for grammatical errors, spelling mistakes, and punctuation inconsistencies. Make notes of any         │
│  awkward phrasing or unclear sections that could benefit from rephrasing.                                       │
│                                                                                                                 │
│  3. Evaluate the structure of each section against the initial outline. Ensure that the sections are coherent,  │
│  with effective transitions that enhance the flow of the post.                                                  │
│                                                                                                                 │
│  4. Confirm that all SEO keywords are integrated subtly and do not disrupt the reading experience. Ensure that  │
│  keyword density adheres to best practices without leading to keyword stuffing.                                 │
│                                                                                                                 │
│  5. Suggest any necessary adjustments to maintain a consistent brand voice throughout the post, whether that    │
│  means altering certain phrases, adopting a specific style, or reinforcing key messaging.                       │
│                                                                                                                 │
│  6. After making corrections, provide a final read-through to ensure the blog post is polished and ready for    │
│  publication, with all adjustments made.                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Unlocking the Future: A Deep Dive into GPT 5.2 and Its Impact on AI Innovation                               │
│                                                                                                                 │
│  ## Introduction: Navigating the New Dawn of Artificial Intelligence                                            │
│                                                                                                                 │
│  In recent years, artificial intelligence has transitioned from a niche technological pursuit to a central      │
│  pillar of modern innovation. The rapid evolution of language models, particularly the GPT series developed by  │
│  OpenAI, exemplifies this exponential growth. As the AI community anticipates the latest iteration—GPT          │
│  5.2—discussions around its potential capabilities, implications, and applications are intensifying. This       │
│  model promises to push the boundaries of what AI can achieve, integrating cutting-edge advancements and        │
│  addressing previous limitations.                                                                               │
│                                                                                                                 │
│  Current trends highlight the increasing adoption of AI across various industries—ranging from healthcare and   │
│  finance to entertainment and education. As businesses and developers explore the capabilities of GPT 5.2,      │
│  understanding its features, improvements, and potential challenges becomes essential. Whether you're a         │
│  technology enthusiast, a business leader, or a researcher, staying informed about GPT 5.2 is crucial for       │
│  harnessing its full potential. This article aims to unpack the significance of GPT 5.2, exploring why it       │
│  matters now more than ever, and what future innovations it may unlock in the realm of AI.                      │
│                                                                                                                 │
│  ## The Evolution of GPT Models: From GPT-3 to GPT 5.2                                                          │
│                                                                                                                 │
│  The journey from GPT-3 to GPT 5.2 reflects a trajectory of continuous enhancement driven by advancements in    │
│  neural network architecture, training data scale, and computational power. GPT-3, introduced in 2020, set a    │
│  new benchmark with 175 billion parameters, demonstrating exceptional language comprehension and generation     │
│  capabilities. Its release enabled widespread integration into chatbots, content creation tools, and            │
│  automation solutions but also revealed limitations such as contextual misunderstandings and biases.            │
│                                                                                                                 │
│  With the launch of GPT 4, expectations surged. This version showed improvements in natural language            │
│  processing (NLP), including enhanced contextual awareness and reduced biases, due to more refined training     │
│  techniques and larger datasets. GPT 5.0 further accelerated this progress by introducing multimodal            │
│  capabilities—integrating text with images and sound—an

In [24]:
from IPython.display import Markdown, display
display(Markdown(result.raw if hasattr(result, "raw") else str(result)))

# Unlocking the Future: A Deep Dive into GPT 5.2 and Its Impact on AI Innovation

## Introduction: Navigating the New Dawn of Artificial Intelligence

In recent years, artificial intelligence has transitioned from a niche technological pursuit to a central pillar of modern innovation. The rapid evolution of language models, particularly the GPT series developed by OpenAI, exemplifies this exponential growth. As the AI community anticipates the latest iteration—GPT 5.2—discussions around its potential capabilities, implications, and applications are intensifying. This model promises to push the boundaries of what AI can achieve, integrating cutting-edge advancements and addressing previous limitations.

Current trends highlight the increasing adoption of AI across various industries—ranging from healthcare and finance to entertainment and education. As businesses and developers explore the capabilities of GPT 5.2, understanding its features, improvements, and potential challenges becomes essential. Whether you're a technology enthusiast, a business leader, or a researcher, staying informed about GPT 5.2 is crucial for harnessing its full potential. This article aims to unpack the significance of GPT 5.2, exploring why it matters now more than ever, and what future innovations it may unlock in the realm of AI.

## The Evolution of GPT Models: From GPT-3 to GPT 5.2

The journey from GPT-3 to GPT 5.2 reflects a trajectory of continuous enhancement driven by advancements in neural network architecture, training data scale, and computational power. GPT-3, introduced in 2020, set a new benchmark with 175 billion parameters, demonstrating exceptional language comprehension and generation capabilities. Its release enabled widespread integration into chatbots, content creation tools, and automation solutions but also revealed limitations such as contextual misunderstandings and biases.

With the launch of GPT 4, expectations surged. This version showed improvements in natural language processing (NLP), including enhanced contextual awareness and reduced biases, due to more refined training techniques and larger datasets. GPT 5.0 further accelerated this progress by introducing multimodal capabilities—integrating text with images and sound—and moving closer toward general intelligence aspirations. GPT 5.2, the latest iteration, builds upon this foundation, emphasizing stability, deeper contextual understanding, and ethical considerations.

What makes GPT 5.2 particularly significant? Its development prioritizes addressing previous shortcomings, such as hallucination errors—confidently generating inaccurate information—and improving alignment with human values. Additionally, GPT 5.2’s architecture is optimized for faster processing, lower latency, and resource efficiency, making it more accessible for practical applications. This evolutionary step marks an important milestone, demonstrating that GPT series is not merely scaling up but also becoming smarter, safer, and more versatile.

## Key Features and Technological Breakthroughs in GPT 5.2

GPT 5.2 introduces several groundbreaking features that set it apart from its predecessors, reflecting the latest in AI research and engineering. A primary advancement is its improved contextual understanding. Unlike earlier models, GPT 5.2 can maintain coherence across longer interactions and complex tasks, reducing issues like drift or inconsistent responses. This enhancement makes it well-suited for applications requiring high precision, such as legal analysis or medical diagnostics.

Another notable feature is GPT 5.2’s enhanced multi-turn reasoning ability. This allows the model to process complex, multi-step problems more effectively, which is valuable in coding, mathematical calculations, and nuanced social interactions. Its training incorporated more diverse datasets, helping to diminish biases and promote fairness—an ongoing challenge in AI development. As a result, GPT 5.2 demonstrates a more balanced, ethical AI performance aligned with global efforts to mitigate potential harm.

Furthermore, GPT 5.2 advances computational efficiency through innovations like sparse attention mechanisms and optimization algorithms. These enable it to produce high-quality outputs with significantly reduced energy consumption and faster response times. Such technological progress democratizes access to advanced AI, allowing startups and smaller enterprises to incorporate powerful language models without prohibitive infrastructural costs. Overall, GPT 5.2 is positioned as a versatile, ethical, and efficient tool for developers and organizations alike.

## Applications: Transforming Industries with GPT 5.2

The potential applications of GPT 5.2 across various sectors are transformative. Its sophisticated language capabilities facilitate more accurate, context-aware interactions, revolutionizing customer service, healthcare, content creation, and beyond. For example, businesses can deploy GPT 5.2-enabled chatbots capable of understanding complex queries and providing human-like, reliable responses—improving engagement and satisfaction.

In healthcare, GPT 5.2’s multi-step reasoning ability can assist medical professionals by synthesizing extensive medical literature, suggesting diagnosis pathways, and supporting personalized treatment plans. Its commitment to ethical and bias-mitigated outputs ensures sensitive information is handled responsibly, aligning with compliance standards. Similarly, the financial industry can leverage GPT 5.2 for real-time analysis, risk assessment, and report generation—streamlining decision-making processes.

Content creators and marketers stand to benefit as well, able to generate high-quality, nuanced content at scale. Whether it’s crafting detailed articles, engaging social media posts, or personalized marketing emails, GPT 5.2 enables faster, consistent, and compelling content production. Its versatility accelerates digital marketing strategies, reduces operational costs, and fosters innovation. Moreover, GPT 5.2 is poised to enhance educational tools through personalized tutoring and adaptive learning experiences, further demonstrating its broad potential to impact various facets of society.

## Ethical Considerations and Challenges Ahead

While GPT 5.2 showcases impressive capabilities, it also raises important ethical considerations. Despite efforts to reduce bias, AI models can inadvertently perpetuate stereotypes or misinformation. Ongoing training improvements aim to mitigate these issues, but complete neutrality remains a challenge. Ensuring the responsible deployment of GPT 5.2 involves transparency, oversight, and adherence to ethical standards.

AI misuse is another concern. Powerful language models can be exploited to generate misinformation, deepfakes, or malicious content. Establishing robust guidelines, transparency measures, and continuous monitoring is essential to prevent harm. OpenAI’s focus on safety and ethics in GPT 5.2’s development underscores the importance of integrating these principles into all stages of AI innovation.

Environmental sustainability is also a critical issue. Large-scale AI training consumes substantial energy, raising concerns about ecological impact. Although GPT 5.2 incorporates resource-efficient features, the industry must prioritize eco-friendly development practices, such as optimizing training processes and leveraging edge computing to reduce reliance on centralized data centers.

Lastly, societal implications—like employment disruption, privacy, and human-AI interactions—necessitate thoughtful regulation and stakeholder collaboration. As GPT 5.2 becomes more advanced and human-like, the importance of developing frameworks that protect individual rights while fostering innovation becomes paramount. Addressing these challenges with a balanced approach is crucial for shaping an ethically responsible AI future.

## The Road Ahead: GPT 5.2’s Role in Shaping the Future

Looking forward, GPT 5.2 embodies more than an incremental update; it represents a milestone in the ongoing journey toward smarter, safer, and more human-aligned AI systems. Its technological advances open doors for applications such as autonomous systems, virtual assistants, and AI-supported research. As AI integration deepens, we can anticipate more seamless, intuitive human-computer interactions that enhance productivity and creativity.

The future of GPT models depends on responsible innovation—balancing advances with ethical obligations. OpenAI’s commitment to transparency and safety signals a positive direction, but widespread adoption will require collaboration among developers, policymakers, and users. GPT 5.2 could catalyze breakthroughs across disciplines, fostering joint insights from linguistics, cognitive science, and computer engineering to refine AI further.

As these models become incorporated into daily life, ongoing debates about employment, privacy, and societal impacts will shape the regulatory landscape. Ensuring equitable benefits from AI’s advancements is a shared responsibility. Ultimately, GPT 5.2 signifies a pivotal step toward a future where AI serves humanity’s best interests—combining innovation with ethical stewardship.

## Join the Conversation

What are your thoughts on GPT 5.2 and its potential to revolutionize AI? Are you excited about the possibilities it offers across your industry or personal projects? We invite you to share your insights, questions, or experiences with GPT models in the comments below. For those eager to stay updated on AI breakthroughs, subscribe to our newsletter for the latest news on GPT developments, ethical AI practices, and innovative applications. Together, let’s shape an AI-powered future that benefits us all.

*Note: This article reflects current insights based on data up to October 2023. As GPT 5.2 and related technologies continue evolving, staying informed through credible sources is essential.*

# Deploy this crew in gradio

In [ ]:
# pip install gradio

In [ ]:
import gradio as gr
from crewai import Agent, Task, Crew

# Define Agents
researcher = Agent(
    role="Senior Research Specialist",
    goal="Uncover intricate insights into the given topic",
    backstory="A seasoned expert in extracting valuable information and providing detailed analysis on any given subject.",
    allow_delegation=False
)

writer = Agent(
    role="Content Writer",
    goal="Craft engaging and informative articles",
    backstory="An experienced content writer who can turn complex research into easy-to-understand and engaging articles.",
    allow_delegation=False
)

editor = Agent(
    role="Content Editor",
    goal="Ensure articles are polished, coherent, and engaging",
    backstory="An expert editor with a sharp eye for clarity, grammar, and flow, capable of turning drafts into publication-ready articles.",
    allow_delegation=False
)

# Function to setup tasks and run CrewAI
def run_crewai_article_writer(topic):
    # Define tasks
    research_task = Task(
        description=f"Conduct in-depth research on the topic: {topic} and provide a structured summary.",
        expected_output="A structured and detailed research summary covering important aspects, recent developments, and insights related to the topic.",
        agent=researcher
    )

    writing_task = Task(
        description=f"Using the research, write a full-length article on the topic: {topic} in 1500 words give various sub section",
        expected_output="An engaging, detailed, informative, and coherent article suitable for online publication.",
        agent=writer
    )

    editing_task = Task(
        description=f"Review and edit the article on {topic} for clarity, grammar, flow, and engagement.",
        expected_output="A polished, clear, and professionally edited final version of the article.",
        agent=editor
    )

    # Setup Crew
    crew = Crew(
        agents=[researcher, writer, editor],
        tasks=[research_task, writing_task, editing_task],
        verbose=True  # Set to False if you want less console output
    )

    # Execute
    result = crew.kickoff()
    return result

# Gradio Interface
def generate_article(topic):
    return run_crewai_article_writer(topic)

iface = gr.Interface(
    fn=generate_article,
    inputs=gr.Textbox(lines=2, placeholder="Enter a topic for the article..."),
    outputs=gr.Textbox(lines=20, label="Generated Article"),
    title="AI Agent: Prompt to Post",
    description="Enter a topic and let CrewAI agents research, write, and edit a complete article!"
)

if __name__ == "__main__":
    iface.launch()
